In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

from Generator import Generator

## Parameters

#### Data Generation

In [2]:
low = 0.1                                    #
high = 5                                     #
rank = True                                  #
normalize = False

train_examples = 10000
test_examples = 100

#### Model

In [3]:
embed_dim = 512                              #
stack = 8                                    #

optimizer = keras.optimizers.Nadam()         #
loss = keras.losses.MSE                      # Regression Loss Function


metrics = keras.metrics.MeanSquaredError()

2022-04-15 00:28:42.924067: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-15 00:28:42.981642: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-15 00:28:42.982319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-15 00:28:42.983415: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

#### Training

In [4]:
batch_size = 128
epochs = 100

validation_split = 0.2
verbose = 2

## Generate Data

In [5]:
generator = Generator('nutrients.csv') # filename with data

In [6]:
generator.generate(train_examples) # generates arg number of examples
generator.scale(low,high) # scales ingredients individually between min and max arg values
if rank:
    generator.rank()
    
if normalize:
    generator.normalize()

## Build Model

In [7]:
embed_dim = embed_dim
stack = stack
i1 = keras.layers.Input(generator.recipes.shape[1:])
f1 = keras.layers.Flatten()(i1)
y1 = keras.layers.Dense(embed_dim)(f1)

# Stack residual blocks (skip connections)
for _ in range(stack):
    y2 = y1
    y1 = keras.layers.Dense(embed_dim,
        activation=keras.activations.relu)(y1)
    y1 = keras.layers.Add()([y1,y2])
    y1 = keras.layers.LayerNormalization()(y1)

if normalize:
    o1 = keras.layers.Dense(generator.recipes.shape[1],activation=keras.activations.softplus)(y1)
    
else:
    o1 = keras.layers.Dense(generator.recipes.shape[1])(y1)
    
model = keras.Model(i1,o1)

In [8]:
#keras.utils.plot_model(model,to_file='recipe_model.png',show_shapes=True,expand_nested=True)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15, 14)]     0           []                               
                                                                                                  
 flatten (Flatten)              (None, 210)          0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 512)          108032      ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 512)          262656      ['dense[0][0]']                  
                                                                                              

In [10]:
model.compile(optimizer=optimizer,
            loss=loss,
            metrics=metrics)

## Train

In [11]:
history = model.fit(
    generator.recipes, generator.quant, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split = validation_split, 
    verbose=verbose
)

Epoch 1/100


2022-04-15 00:28:49.131864: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-04-15 00:28:49.555486: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301


63/63 - 5s - loss: 1.1358 - mean_squared_error: 1.1358 - val_loss: 0.6017 - val_mean_squared_error: 0.6017 - 5s/epoch - 87ms/step
Epoch 2/100
63/63 - 1s - loss: 0.5349 - mean_squared_error: 0.5349 - val_loss: 0.7974 - val_mean_squared_error: 0.7974 - 773ms/epoch - 12ms/step
Epoch 3/100
63/63 - 1s - loss: 0.4186 - mean_squared_error: 0.4186 - val_loss: 0.3889 - val_mean_squared_error: 0.3889 - 759ms/epoch - 12ms/step
Epoch 4/100
63/63 - 1s - loss: 0.3230 - mean_squared_error: 0.3230 - val_loss: 0.3371 - val_mean_squared_error: 0.3371 - 780ms/epoch - 12ms/step
Epoch 5/100
63/63 - 1s - loss: 0.2709 - mean_squared_error: 0.2709 - val_loss: 0.3341 - val_mean_squared_error: 0.3341 - 776ms/epoch - 12ms/step
Epoch 6/100
63/63 - 1s - loss: 0.2493 - mean_squared_error: 0.2493 - val_loss: 0.2983 - val_mean_squared_error: 0.2983 - 817ms/epoch - 13ms/step
Epoch 7/100
63/63 - 1s - loss: 0.2395 - mean_squared_error: 0.2395 - val_loss: 0.2996 - val_mean_squared_error: 0.2996 - 820ms/epoch - 13ms/step


## Testing

In [12]:
#prediction values
test_gen = Generator('nutrients.csv')
test_gen.generate(test_examples) # generates arg number of examples
test_gen.scale(low,high) # scales ingredients individually between min and max arg values

if rank:
    test_gen.rank()
    
if normalize:
    test_gen.normalize()

In [13]:
test_recipes = test_gen.recipes
test_quant = test_gen.quant

In [14]:
score = model.evaluate(test_recipes, test_quant, verbose=1)

4/4 [==============================] - 0s 3ms/step - loss: 0.2327 - mean_squared_error: 0.2327


In [15]:
pred = model.predict(test_recipes)
if normalize:
    pred = test_gen.inv_normalize(pred)

In [16]:
actual = test_gen.quant
if normalize:
    actual = test_gen.inv_normalize(actual)

In [23]:
index = 4

In [27]:
for i in range(len(pred[0])):
    pred_num = str(round(pred[index,i],3))
    act_num = str(round(actual[index,i],3))
    
    print(f"pred: {pred_num} | actual: {act_num}" )

pred: -0.239 | actual: 0.0
pred: 2.535 | actual: 3.597
pred: 0.429 | actual: 1.333
pred: 0.992 | actual: 1.362
pred: 1.912 | actual: 2.185
pred: 2.128 | actual: 2.244
pred: 1.894 | actual: 1.911
pred: -0.34 | actual: 0.0
pred: 1.864 | actual: 1.597
pred: 1.574 | actual: 1.735
pred: 2.532 | actual: 2.421
pred: 4.656 | actual: 4.292
pred: 0.444 | actual: 0.755
pred: -0.005 | actual: 0.0
pred: -0.256 | actual: 0.0


In [22]:
round(pred[index,i],3)

2.431

In [25]:
pred_num = round(pred[index,i],3)

In [26]:
pred_num

-0.256